In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

from sklearn.pipeline import make_pipeline
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation

import json
import random

import tkinter as tk
from tkinter import Text, Scrollbar, Button

## Data Preprocessing

In [8]:
# Process text
def process_text(text):
    # Lowercasing
    text = text.lower()
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing Punctuation
    tokens = [token for token in tokens if token not in punctuation]
    
    # Stopword Removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatization and Stemming
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(lemmatized_tokens)

## Load The Data

In [9]:
# Load intent data from JSON
with open('./intents.json', 'r') as file:
    data = json.load(file)

# Prepare data for training
patterns = []
responses = []
for intent in data['intents']:
    processed_patterns = [process_text(pattern) for pattern in intent['patterns']]
    patterns.extend(processed_patterns)
    responses.extend([intent['tag']] * len(intent['patterns']))

## Model Of Predict

In [10]:
# Create a TF-IDF vectorizer and a Support Vector Machine classifier
model = make_pipeline(TfidfVectorizer(), SVC())

# Train the model
model.fit(patterns, responses)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()), ('svc', SVC())])

# Test Our Chatbot

In [11]:
# Test the model
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit', 'bye']:
        break

    # Process user input
    processed_input = process_text(user_input)

    # Predict the intent
    predicted_intent = model.predict([processed_input])

    # Find the corresponding responses for the intent
    intent_responses = []
    for intent in data['intents']:
        if intent['tag'] == predicted_intent[0]:
            intent_responses = intent['responses']
            break

    # Choose a random response
    response = random.choice(intent_responses) if intent_responses else "Sorry, I don't understand."

    # Print the response
    print("Bot:", response)

Bot: Hi there, how can I help?


# GUI With Tkinter

<img src='./1.jpg'>

In [12]:
 class SimpleChatbotGUI:
    def __init__(self):
        self.base = tk.Tk()
        self.base.title("AI Eagels")
        self.base.geometry("600x400")
        self.base.resizable(width=False, height=False)

        self.ChatLog = Text(self.base, bd=1, bg="#2c2c2c", fg="#ffffff", insertbackground="#ffffff",
                            height="15", width="60", font="Arial")
        self.ChatLog.config(state=tk.DISABLED)

        self.scrollbar = Scrollbar(self.base, command=self.ChatLog.yview, cursor="heart")
        self.ChatLog['yscrollcommand'] = self.scrollbar.set

        self.SendButton = Button(self.base, font=("Verdana", 12, 'bold'), text="Send", width="8", height=2,
                                 bd=0, bg="#32de97", activebackground="#3c9d9b", fg='#ffffff',
                                 command=self.send_message)

        self.EntryBox = Text(self.base, bd=0, bg="#2c2c2c", fg="#ffffff", insertbackground="#ffffff",
                             width="45", height="2", font="Arial")
        self.EntryBox.bind("<Return>", self.on_enter)  # Bind Enter key
        self.EntryBox.bind("<Escape>", self.on_escape)  # Bind Esc key

        self.scrollbar.place(x=540, y=6, height=286)
        self.ChatLog.place(x=10, y=6, height=286, width=520)
        self.EntryBox.place(x=10, y=306, height=80, width=470)
        self.SendButton.place(x=490, y=306, height=80)

    def send_message(self, event=None):
        user_input = self.EntryBox.get("1.0", 'end-1c').strip()
        self.EntryBox.delete("0.0", tk.END)

        if user_input != '':
            self.ChatLog.config(state=tk.NORMAL)
            self.ChatLog.insert(tk.END, "User : " + user_input + '\n\n')
            self.ChatLog.config(foreground="#ffffff", font=("Verdana", 12))

            # Process user input
            processed_input = process_text(user_input)

            # Predict the intent
            predicted_intent = model.predict([processed_input])

            # Find the corresponding responses for the intent
            intent_responses = []
            for intent in data['intents']:
                if intent['tag'] == predicted_intent[0]:
                    intent_responses = intent['responses']
                    break

            # Choose a random response
            response = random.choice(intent_responses) if intent_responses else "Sorry, I don't understand."

            self.ChatLog.insert(tk.END, "AI Eagles Bot : " + response + '\n\n')

            self.ChatLog.config(state=tk.DISABLED)
            self.ChatLog.yview(tk.END)

    def on_enter(self, event):
        self.send_message()

    def on_escape(self, event):
        self.base.destroy()

    def run(self):
        self.base.mainloop()

# Create an instance of SimpleChatbotGUI
simple_chatbot_gui = SimpleChatbotGUI()

# Run the GUI
simple_chatbot_gui.run()

# Thanks